In [194]:
import polars as pl
from polars import col as c
import networkx as nx

from config import settings
import json
import os
from datetime import datetime, UTC
import datetime as dt

from utility.polars_operation import generate_uuid_col
from utility.parser_utility import (
    add_table_to_changes_schema,
    generate_connectivity_table,
    generate_random_uuid,
)
from utility.general_function import pl_to_dict

from twindigrid_changes.schema import ChangesSchema
from twindigrid_sql.schema.enum import (
    MeasurementClass,
    MeasurementPhase,
    MeasurementColumn,
    SubstationType,
    TerminalSide,
)
from twindigrid_sql.entries.source import (
    SCADA,
    CONVENTIONAL_METER,
    GRID_LOAD,
    SCADA,
    ESTIMATED,
)


from twindigrid_sql.entries.equipment_class import (
    TRANSFORMER,
    BRANCH,
    SWITCH,
    INDIRECT_FEEDER,
    BUSBAR_SECTION,
    ENERGY_CONSUMER,
)
from twindigrid_sql.entries.measurement_type import ENERGY, ACTIVE_POWER, REACTIVE_POWER
from twindigrid_sql.entries.unit_symbol import WATTHOUR, WATT

# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))
# os.getcwd()

# Import data from matlab

In [195]:
file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))

In [196]:
parameter_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_parameter"])
nodedata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_node_data"])
powerdata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_Power_data"])
linedata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_Line_data"])
result_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_result"])
# nodedata_distflow = nodedata_distflow.with_columns(c("Snom").cast(pl.Int8))
# # To have base value (need lenght of line), not from matlab !
# line_data_from_input_file: pl.DataFrame = pl.read_excel(
#     file_names["Line_Data_From_Input_File"]
# )

# Add node number to power data
powerdata_distflow = powerdata_distflow.with_row_index(
    "node_number", offset=1
)  # offset=1 because slack bus is 0 and no power on it
powerdata_distflow = powerdata_distflow.with_columns(c("node_number").cast(pl.Int64))
# Create a topology dataframe with basic topology information

df_topology = nodedata_distflow.select(
    c("index").alias("node_number"),
    c("Vnom"),
)

# Add the power data to the topology dataframe with node as key
df_topology = df_topology.join(
    powerdata_distflow, on="node_number", how="full", coalesce=True
)
powerdata_distflow

node_number,Pload,Qload
i64,f64,f64
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.000035,0.000008
…,…,…
53,0.0,0.0
54,0.000035,-0.000012
55,0.0,0.0


# Set missing value for equipment

In [197]:
### Set missing value for equipment
# Fake value for the length of the branch
base_length = 1
# Fake value for the switch state
switch_state = False
switch_type = "locked_switch"
switch_command = "unknown"

## Connectivity node table

In [198]:
# Generate the node dict with uuid for each node
connectivity_node: dict[float, str] = pl_to_dict(
    df_topology.select(
        c("node_number"),
        c("node_number").pipe(generate_uuid_col, added_string="node_").alias("uuid"),
    )
)
## Add the cn_fk to the topology dataframe
df_topology = df_topology.with_columns(
    c("node_number").replace_strict(connectivity_node, default=None).alias("cn_fk")
)

## Branch

In [199]:
# branch :pl.DataFrame =

default_install_date: datetime = datetime(*settings.DEFAULT_INSTALL_DATE, tzinfo=UTC)
heartbeat = datetime.now(UTC)
changes_schema = ChangesSchema()


# Current and other line parameter in pu

# Filter to take only branch, connection_type == 2
branch = linedata_distflow.filter(c("connection_type") == 2).select(
    c("line_number").alias("dso_code"),
    c("i_pu").alias("current_limit"),
    c("r_pu"),
    c("x_pu"),
    c("b_pu"),
    # Need column name non null value for validation of the schema
    pl.lit(base_length).alias("length"),  # km
    pl.lit(BRANCH).alias("concrete_class"),
    pl.lit(default_install_date).alias("start"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    c("line_number").pipe(generate_uuid_col, added_string=BRANCH).alias("uuid"),
    # Generate uuid for each terminal of branch with node uuid
    c("node_from").replace_strict(connectivity_node, default=None).alias("t1"),
    c("node_to").replace_strict(connectivity_node, default=None).alias("t2"),
    # Need column name for validation of the schema
    pl.lit(None).alias("t1_container_fk"),
    pl.lit(None).alias("t2_container_fk"),
)
new_tables_pl: dict[str, pl.DataFrame] = {
    "Resource": branch,
    "Equipment": branch,
    "Branch": branch,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="branch"
)
changes_schema = generate_connectivity_table(
    changes_schema=changes_schema, eq_table=branch, raw_data_table="branch"
)

## Energy consumer

In [200]:
from polars import Null


default_install_date: datetime = datetime(*settings.DEFAULT_INSTALL_DATE, tzinfo=UTC)
heartbeat = datetime.now(UTC)
changes_schema = ChangesSchema()


# Power in PU
energy_consumer = df_topology.with_columns(
    pl.lit(ENERGY_CONSUMER).alias("concrete_class"),
    pl.lit(default_install_date).alias("start"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit("unknown").alias("profile_type"),
    pl.lit(0).alias("rated_p"),  # Symbol: P_ec_nom, Unit: kW
    c("cn_fk")
    .pipe(generate_random_uuid)
    .alias("uuid"),  # Generate random uuid on a random column
    c("Pload").replace(0, None).alias("node_with_consumer"),
).drop_nulls(
    "node_with_consumer"
)  # Remove node without consumption
new_tables_pl: dict[str, pl.DataFrame] = {
    "Resource": energy_consumer,
    "Equipment": energy_consumer,
    "EnergyConsumer": energy_consumer,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="energy_consumer"
)
changes_schema = generate_connectivity_table(
    changes_schema=changes_schema, eq_table=branch, raw_data_table="energy_consumer"
)

## Measurement

In [201]:
energy_consumer

node_number,Vnom,Pload,Qload,cn_fk,concrete_class,start,start_heartbeat,profile_type,rated_p,uuid,node_with_consumer
i64,i64,f64,f64,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",str,i32,str,f64
5,400,0.000035,0.000008,"""dbd2411e-1e87-5956-86d9-d69ee7…","""energy_consumer""",1960-01-01 00:00:00 UTC,2025-01-10 08:36:26.683519 UTC,"""unknown""",0,"""e0afa9c2-2997-44e6-be33-1aa4cd…",0.000035
8,400,0.001251,-0.000375,"""d177af44-109b-50d6-8b8a-97874a…","""energy_consumer""",1960-01-01 00:00:00 UTC,2025-01-10 08:36:26.683519 UTC,"""unknown""",0,"""885ff105-fac6-4fb2-96a1-b82fdc…",0.001251
9,400,0.002761,-0.000914,"""8a7f105e-71f3-5101-8b4f-1a9007…","""energy_consumer""",1960-01-01 00:00:00 UTC,2025-01-10 08:36:26.683519 UTC,"""unknown""",0,"""6474caa2-642f-4cff-b70e-155aab…",0.002761
11,400,0.000257,-0.000084,"""2974cbe7-7e8b-54b5-9978-65f421…","""energy_consumer""",1960-01-01 00:00:00 UTC,2025-01-10 08:36:26.683519 UTC,"""unknown""",0,"""13968615-4b3a-4158-9c51-46352b…",0.000257
13,400,0.001223,-0.000406,"""2e63d367-00e4-59d5-a861-9117f6…","""energy_consumer""",1960-01-01 00:00:00 UTC,2025-01-10 08:36:26.683519 UTC,"""unknown""",0,"""4daa3b6b-1e75-49c5-8ab0-ed9547…",0.001223
…,…,…,…,…,…,…,…,…,…,…,…
47,400,0.000041,-0.000013,"""2023dca7-8cab-5a99-bb1c-926575…","""energy_consumer""",1960-01-01 00:00:00 UTC,2025-01-10 08:36:26.683519 UTC,"""unknown""",0,"""20becc2b-7a39-48da-ba78-372f82…",0.000041
51,400,0.000105,-0.000034,"""fb87ec61-e87a-5861-88bf-d2d23f…","""energy_consumer""",1960-01-01 00:00:00 UTC,2025-01-10 08:36:26.683519 UTC,"""unknown""",0,"""ea320a0f-c718-41b6-af8d-e386f7…",0.000105
52,400,0.000034,-0.000011,"""d2deff68-20a8-5d92-b689-a57381…","""energy_consumer""",1960-01-01 00:00:00 UTC,2025-01-10 08:36:26.683519 UTC,"""unknown""",0,"""a0a76f9f-0c6a-4a69-acf9-ef67a8…",0.000034


Active power

In [202]:
## Add the uuid of the node to the power data
measurement = energy_consumer.select(
    c("uuid")
    .pipe(generate_random_uuid)
    .alias(
        "uuid"
    ),  # Generate random uuid on a column without importance (don't work with pl.lit)
    c("uuid").alias("resource_fk"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit(MeasurementClass.SPAN.value).alias("concrete_type"),
    pl.lit(MeasurementPhase.ABC.value).alias("phase"),
    pl.lit(MeasurementColumn.DOUBLE.value).alias("column_type"),
    pl.lit(CONVENTIONAL_METER).alias("source_fk"),
    # pl.lit(60*60*24*365).alias("default_period"),
    pl.lit(ACTIVE_POWER).alias("measurement_type"),
    pl.lit("pu").alias("unit_symbol"),
    pl.lit(1).alias("unit_multiplier"),
    c("Pload").alias("double_value"),
)
measurement

uuid,resource_fk,start_heartbeat,concrete_type,phase,column_type,source_fk,measurement_type,unit_symbol,unit_multiplier,double_value
str,str,"datetime[μs, UTC]",str,str,str,str,str,str,i32,f64
"""f612fc65-91dd-4a05-a751-d00a00…","""e0afa9c2-2997-44e6-be33-1aa4cd…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.000035
"""1900bc3c-3169-431c-8449-c934dd…","""885ff105-fac6-4fb2-96a1-b82fdc…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.001251
"""14e94576-b365-431b-b242-c3c6b8…","""6474caa2-642f-4cff-b70e-155aab…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.002761
"""0dd3f6f2-6728-46f6-aa29-d943d7…","""13968615-4b3a-4158-9c51-46352b…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.000257
"""158445bc-ec2e-4243-8639-fc216e…","""4daa3b6b-1e75-49c5-8ab0-ed9547…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.001223
…,…,…,…,…,…,…,…,…,…,…
"""576325c7-c8df-4307-b1c8-76bca7…","""20becc2b-7a39-48da-ba78-372f82…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.000041
"""f5e7b1e9-1d5a-4c91-b8d5-7beaf8…","""ea320a0f-c718-41b6-af8d-e386f7…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.000105
"""b066152b-8006-448e-95e4-bb8559…","""a0a76f9f-0c6a-4a69-acf9-ef67a8…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.000034


In [203]:
measurement_span = measurement.with_columns(
    c("uuid").alias("measurement_fk"),
    c("uuid").pipe(generate_random_uuid).alias("uuid"),
    pl.lit(datetime(2022, 1, 1))
    .dt.replace_time_zone(time_zone="Europe/Zurich")
    .dt.convert_time_zone(time_zone="UTC")
    .alias("start"),
    pl.lit(datetime(2023, 1, 1))
    .dt.replace_time_zone(time_zone="Europe/Zurich")
    .dt.convert_time_zone(time_zone="UTC")
    .alias("end"),
)

new_tables_pl: dict[str, pl.DataFrame] = {
    "Measurement": measurement,
    "MeasurementSpan": measurement_span,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="meter_id"
)

In [204]:
measurement

uuid,resource_fk,start_heartbeat,concrete_type,phase,column_type,source_fk,measurement_type,unit_symbol,unit_multiplier,double_value
str,str,"datetime[μs, UTC]",str,str,str,str,str,str,i32,f64
"""f612fc65-91dd-4a05-a751-d00a00…","""e0afa9c2-2997-44e6-be33-1aa4cd…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.000035
"""1900bc3c-3169-431c-8449-c934dd…","""885ff105-fac6-4fb2-96a1-b82fdc…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.001251
"""14e94576-b365-431b-b242-c3c6b8…","""6474caa2-642f-4cff-b70e-155aab…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.002761
"""0dd3f6f2-6728-46f6-aa29-d943d7…","""13968615-4b3a-4158-9c51-46352b…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.000257
"""158445bc-ec2e-4243-8639-fc216e…","""4daa3b6b-1e75-49c5-8ab0-ed9547…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.001223
…,…,…,…,…,…,…,…,…,…,…
"""576325c7-c8df-4307-b1c8-76bca7…","""20becc2b-7a39-48da-ba78-372f82…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.000041
"""f5e7b1e9-1d5a-4c91-b8d5-7beaf8…","""ea320a0f-c718-41b6-af8d-e386f7…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.000105
"""b066152b-8006-448e-95e4-bb8559…","""a0a76f9f-0c6a-4a69-acf9-ef67a8…",2025-01-10 08:36:26.683519 UTC,"""measurement_span""","""ABC""","""double_value""","""conventional_meter""","""active power""","""pu""",1,0.000034


Reactive power

In [205]:
# changes_schema.connectivity
# changes_schema.measurement["resource_fk"][0]
# changes_schema.branch.filter(c("uuid") == "df941fce-ceda-5874-ab63-5c8af9bec38b")
# changes_schema.connectivity.filter(
#     c("cn_fk").is_in(changes_schema.measurement["resource_fk"])
# )
changes_schema.measurement

diff,uuid,start_heartbeat,end_heartbeat,concrete_type,resource_fk,terminal_side,phase,measurement_type,unit_multiplier,unit_symbol,column_type,source_fk,sensor_accuracy,name,description,metadata,op_type,over_measurement_fk,default_period
str,str,"datetime[μs, UTC]","datetime[μs, UTC]",str,str,str,str,str,i32,str,str,str,f64,str,str,str,str,str,i32
"""+""","""f612fc65-91dd-4a05-a751-d00a00…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""e0afa9c2-2997-44e6-be33-1aa4cd…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""1900bc3c-3169-431c-8449-c934dd…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""885ff105-fac6-4fb2-96a1-b82fdc…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""14e94576-b365-431b-b242-c3c6b8…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""6474caa2-642f-4cff-b70e-155aab…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""0dd3f6f2-6728-46f6-aa29-d943d7…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""13968615-4b3a-4158-9c51-46352b…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""158445bc-ec2e-4243-8639-fc216e…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""4daa3b6b-1e75-49c5-8ab0-ed9547…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""+""","""576325c7-c8df-4307-b1c8-76bca7…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""20becc2b-7a39-48da-ba78-372f82…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""f5e7b1e9-1d5a-4c91-b8d5-7beaf8…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""ea320a0f-c718-41b6-af8d-e386f7…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""b066152b-8006-448e-95e4-bb8559…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""a0a76f9f-0c6a-4a69-acf9-ef67a8…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null


In [206]:
list(changes_schema.__dict__.keys())

['heartbeat',
 'resource',
 'equipment',
 'terminal',
 'busbar_section',
 'branch',
 'branch_parameter_event',
 'geo_event',
 'switch',
 'switch_event',
 'transformer',
 'transformer_end',
 'transformer_parameter_event',
 'tap',
 'tap_event',
 'bess',
 'energy_consumer',
 'external_network',
 'generating_unit',
 'container',
 'client',
 'substation',
 'base_voltage',
 'connectivity_node',
 'connectivity',
 'measurement',
 'measurement_point',
 'measurement_span']

## Switch

In [207]:
# Filter to take only switch, connection_type == 3
switch = linedata_distflow.filter(c("connection_type") == 3).select(
    c("line_number").alias("dso_code"),
    pl.lit(SWITCH).alias("concrete_class"),
    pl.lit(default_install_date).alias("start"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit(switch_state).alias("normal_open"),
    pl.lit(switch_type).alias("type"),
    pl.lit(switch_command).alias("command"),
    # Generate uuid for each terminal of branch with node uuid
    c("node_from").replace_strict(connectivity_node, default=None).alias("t1"),
    c("node_to").replace_strict(connectivity_node, default=None).alias("t2"),
    # Need column name for validation of the schema
    pl.lit(None).alias("t1_container_fk"),
    pl.lit(None).alias("t2_container_fk"),
    c("line_number").pipe(generate_uuid_col, added_string=SWITCH).alias("uuid"),
)
new_tables_pl: dict[str, pl.DataFrame] = {
    "Resource": switch,
    "Equipment": switch,
    "Switch": switch,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="switch"
)
changes_schema = generate_connectivity_table(
    changes_schema=changes_schema, eq_table=switch, raw_data_table="switch"
)

In [208]:
# Begin time of the data from matlab (from main_FC.ipynb before)
str(datetime(2020, 4, 4, 23, 00, 0, 0, UTC) - dt.timedelta(hours=192))

'2020-03-27 23:00:00+00:00'

## Parser

In [209]:
# Parse connectivity node
df_topology

node_number,Vnom,Pload,Qload,cn_fk
i64,i64,f64,f64,str
0,400,null,null,"""df941fce-ceda-5874-ab63-5c8af9…"
1,400,0.0,0.0,"""ba84d70a-80d7-590e-b112-f9c4b5…"
2,400,0.0,0.0,"""078656ed-79f8-53a1-a67a-bb8f53…"
3,400,0.0,0.0,"""c2247320-9fc2-538a-ba64-3ac70e…"
4,400,0.0,0.0,"""af72457f-f983-5eeb-a635-0609f4…"
…,…,…,…,…
53,400,0.0,0.0,"""4505ed8e-f087-5ee2-8c67-775daa…"
54,400,0.000035,-0.000012,"""23bc00b6-0e27-5e6d-a02e-dda5e9…"
55,400,0.0,0.0,"""b1d51456-8036-5737-accc-1103d2…"


In [210]:
def parse_connectivity_node(
    topology_df: pl.DataFrame, changes_schema: ChangesSchema, **kwargs
) -> ChangesSchema:

    cn_voltage_mapping: dict[str, float] = pl_to_dict(
        topology_df.filter(c("KEYWORD") != "TR2")
        .unpivot(
            index=["UN"], on=["t1", "t2"], value_name="cn_fk", variable_name="side"
        )
        .drop_nulls("cn_fk")
        .group_by("cn_fk")
        .agg(c("UN").drop_nulls().first())
        .drop_nulls("UN")[["cn_fk", "UN"]]
    )
    node = topology_df.filter(c("KEYWORD") == "NODE").with_columns(
        (1e3 * c("uuid").replace_strict(cn_voltage_mapping, default=c("UN")))
        .cast(pl.Int32)
        .alias("base_voltage_fk"),  # kV to V
    )

    changes_schema = add_table_to_changes_schema(
        schema=changes_schema,
        new_tables_pl={"ConnectivityNode": node},
        raw_table_name="ConnectivityNode",
    )
    return changes_schema

## Distflow

In [211]:
list(changes_schema.__dict__.keys())

['heartbeat',
 'resource',
 'equipment',
 'terminal',
 'busbar_section',
 'branch',
 'branch_parameter_event',
 'geo_event',
 'switch',
 'switch_event',
 'transformer',
 'transformer_end',
 'transformer_parameter_event',
 'tap',
 'tap_event',
 'bess',
 'energy_consumer',
 'external_network',
 'generating_unit',
 'container',
 'client',
 'substation',
 'base_voltage',
 'connectivity_node',
 'connectivity',
 'measurement',
 'measurement_point',
 'measurement_span']

In [212]:
## Update Qload due to line capacity
changes_schema.measurement

diff,uuid,start_heartbeat,end_heartbeat,concrete_type,resource_fk,terminal_side,phase,measurement_type,unit_multiplier,unit_symbol,column_type,source_fk,sensor_accuracy,name,description,metadata,op_type,over_measurement_fk,default_period
str,str,"datetime[μs, UTC]","datetime[μs, UTC]",str,str,str,str,str,i32,str,str,str,f64,str,str,str,str,str,i32
"""+""","""f612fc65-91dd-4a05-a751-d00a00…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""e0afa9c2-2997-44e6-be33-1aa4cd…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""1900bc3c-3169-431c-8449-c934dd…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""885ff105-fac6-4fb2-96a1-b82fdc…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""14e94576-b365-431b-b242-c3c6b8…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""6474caa2-642f-4cff-b70e-155aab…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""0dd3f6f2-6728-46f6-aa29-d943d7…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""13968615-4b3a-4158-9c51-46352b…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""158445bc-ec2e-4243-8639-fc216e…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""4daa3b6b-1e75-49c5-8ab0-ed9547…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""+""","""576325c7-c8df-4307-b1c8-76bca7…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""20becc2b-7a39-48da-ba78-372f82…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""f5e7b1e9-1d5a-4c91-b8d5-7beaf8…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""ea320a0f-c718-41b6-af8d-e386f7…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null
"""+""","""b066152b-8006-448e-95e4-bb8559…",2025-01-10 08:36:26.683519 UTC,null,"""measurement_span""","""a0a76f9f-0c6a-4a69-acf9-ef67a8…",null,"""ABC""","""active power""",1,"""pu""","""double_value""","""conventional_meter""",null,null,null,null,null,null,null


## Import data to changes schema

In [213]:
def sum_downstream_power(col: pl.Expr, df: pl.DataFrame):
    return col.map_elements(
        lambda x: df.filter(c("upstream") == x)["p_line"].sum(), return_dtype=pl.Float64
    )


def calculate_line_power(df: pl.DataFrame):
    return (c("downstream").pipe(sum_downstream_power, df=df) + c("P")) * (1 + c("F"))


def sum_power(df: pl.DataFrame, lv: int):

    return df.with_columns(
        pl.when(c("lv") == lv)
        .then(calculate_line_power(df=df))
        .otherwise(c("p_line"))
        .alias("p_line")
    )


# UP Use for each powerflow
# Down Use only one time
def get_node_level(G: nx.DiGraph) -> dict:
    level_mapping: dict = {}
    for node in reversed(list(nx.topological_sort(G))):
        if not len(list(G.successors(node))):
            level_mapping[node] = 0
        else:
            level_mapping[node] = max(level_mapping[n] for n in G.successors(node)) + 1
    return level_mapping


line_data: pl.DataFrame = pl.DataFrame(
    {
        "downstream": [1, 2, 3, 4, 5, 6, 7, 8],
        "upstream": [None, 1, 2, 1, 4, 4, 4, 6],
        "P": [0, 1, 2, 1, 4, 3, 6, 5],
        "F": [0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
        "p_line": [0] * 8,
    }
)

grid = nx.DiGraph()

_ = line_data.drop_nulls(subset="upstream").with_columns(
    pl.struct(c("upstream"), c("downstream")).map_elements(
        lambda x: grid.add_edge(x["upstream"], x["downstream"]), return_dtype=pl.Struct
    )
)
level_mapping: dict = get_node_level(G=grid)
line_data = line_data.with_columns(
    c("downstream").replace_strict(level_mapping, default=None).alias("lv")
)

for i in range(line_data["lv"].max() + 1):
    line_data = sum_power(df=line_data, lv=i)

print(line_data.sort("lv"))

shape: (8, 6)
┌────────────┬──────────┬─────┬─────┬────────┬─────┐
│ downstream ┆ upstream ┆ P   ┆ F   ┆ p_line ┆ lv  │
│ ---        ┆ ---      ┆ --- ┆ --- ┆ ---    ┆ --- │
│ i64        ┆ i64      ┆ i64 ┆ f64 ┆ f64    ┆ i64 │
╞════════════╪══════════╪═════╪═════╪════════╪═════╡
│ 3          ┆ 2        ┆ 2   ┆ 0.1 ┆ 2.2    ┆ 0   │
│ 5          ┆ 4        ┆ 4   ┆ 0.1 ┆ 4.4    ┆ 0   │
│ 7          ┆ 4        ┆ 6   ┆ 0.1 ┆ 6.6    ┆ 0   │
│ 8          ┆ 6        ┆ 5   ┆ 0.1 ┆ 5.5    ┆ 0   │
│ 2          ┆ 1        ┆ 1   ┆ 0.1 ┆ 3.52   ┆ 1   │
│ 6          ┆ 4        ┆ 3   ┆ 0.1 ┆ 9.35   ┆ 1   │
│ 4          ┆ 1        ┆ 1   ┆ 0.1 ┆ 23.485 ┆ 2   │
│ 1          ┆ null     ┆ 0   ┆ 0.0 ┆ 27.005 ┆ 3   │
└────────────┴──────────┴─────┴─────┴────────┴─────┘
